# 請結合前面的知識與程式碼，比較不同的 optimizer 與 learning rate 組合對訓練的結果與影響
### 常見的 optimizer 包含
* SGD
* RMSprop
* AdaGrad
* Adam

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
            
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    
    return model

In [8]:
## 超參數設定
"""
Set your required experiment parameters
"""
LEARNING_RATE = [1e-1, 1e-3, 1e-5]
EPOCHS = 50
BATCH_SIZE = 256
MOMENTUM = 0.95

Optimizer = ['SGD','RMSprop','AdaGrad','Adam']

In [9]:
results = {}
"""
建立你的訓練與實驗迴圈並蒐集資料
"""
for opt_str in Optimizer:
    for lr in LEARNING_RATE:
        keras.backend.clear_session() # 把舊的 Graph 清掉
        print("Experiment with LR = %.6f" % (lr))
        model = build_mlp(input_shape=x_train.shape[1:])
        model.summary()
        
        if opt_str == 'SGD':
            optimizer = keras.optimizers.SGD(lr=lr, nesterov=True, momentum=MOMENTUM)
        elif opt_str =='RMSprop':
            optimizer = keras.optimizers.RMSprop(lr=0.001, epsilon=None, decay=0.0)
        elif opt_str =='AdaGrad':
            optimizer = keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
        elif opt_str =='Adam':
            optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        
        
        model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

        model.fit(x_train, y_train, 
                  epochs=EPOCHS, 
                  batch_size=BATCH_SIZE, 
                  validation_data=(x_test, y_test), 
                  shuffle=True)

        # Collect results
        train_loss = model.history.history["loss"]
        valid_loss = model.history.history["val_loss"]
        train_acc = model.history.history["acc"]
        valid_acc = model.history.history["val_acc"]

        exp_name_tag = "exp-lr-%s-opt-%s" % (str(lr),str(opt_str))
        results[exp_name_tag] = {'train-loss': train_loss,
                                 'valid-loss': valid_loss,
                                 'train-acc': train_acc,
                                 'valid-acc': valid_acc}




Experiment with LR = 0.100000


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



50000/50000 [==============================] - 5s 93us/step - loss: 1.5246 - acc: 0.4609 - val_loss: 1.6762 - val_acc: 0.4060
Epoch 28/50
50000/50000 [==============================] - 5s 92us/step - loss: 1.5204 - acc: 0.4646 - val_loss: 1.6444 - val_acc: 0.4202
Epoch 29/50
50000/50000 [==============================] - 5s 94us/step - loss: 1.5247 - acc: 0.4620 - val_loss: 1.6254 - val_acc: 0.4126
Epoch 30/50
50000/50000 [==============================] - 5s 98us/step - loss: 1.5154 - acc: 0.4659 - val_loss: 1.6229 - val_acc: 0.4223
Epoch 31/50
50000/50000 [==============================] - 6s 118us/step - loss: 1.4940 - acc: 0.4738 - val_loss: 1.6936 - val_acc: 0.4118
Epoch 32/50
50000/50000 [==============================] - 5s 108us/step - loss: 1.4995 - acc: 0.4717 - val_loss: 1.6164 - val_acc: 0.4414
Epoch 33/50
50000/50000 [==============================] - 5s 109us/step - loss: 1.4989 - acc: 0.4701 - val_loss: 1.6551 - val_acc: 0.4291
Epoch 34/50
50000/50000 [==================

50000/50000 [==============================] - 5s 93us/step - loss: 1.1604 - acc: 0.5909 - val_loss: 1.3606 - val_acc: 0.5181
Epoch 29/50
50000/50000 [==============================] - 5s 91us/step - loss: 1.1447 - acc: 0.5973 - val_loss: 1.3563 - val_acc: 0.5258
Epoch 30/50
50000/50000 [==============================] - 5s 92us/step - loss: 1.1309 - acc: 0.6054 - val_loss: 1.3635 - val_acc: 0.5207
Epoch 31/50
50000/50000 [==============================] - 5s 91us/step - loss: 1.1165 - acc: 0.6061 - val_loss: 1.3658 - val_acc: 0.5193
Epoch 32/50
50000/50000 [==============================] - 5s 92us/step - loss: 1.1028 - acc: 0.6133 - val_loss: 1.3720 - val_acc: 0.5172
Epoch 33/50
50000/50000 [==============================] - 5s 91us/step - loss: 1.0925 - acc: 0.6150 - val_loss: 1.3609 - val_acc: 0.5275
Epoch 34/50
50000/50000 [==============================] - 5s 92us/step - loss: 1.0778 - acc: 0.6207 - val_loss: 1.4358 - val_acc: 0.5093
Epoch 35/50
50000/50000 [=====================

50000/50000 [==============================] - 5s 105us/step - loss: 1.9977 - acc: 0.3015 - val_loss: 1.9940 - val_acc: 0.2981
Epoch 30/50
50000/50000 [==============================] - 5s 106us/step - loss: 1.9921 - acc: 0.3030 - val_loss: 1.9886 - val_acc: 0.3026
Epoch 31/50
50000/50000 [==============================] - 5s 102us/step - loss: 1.9868 - acc: 0.3048 - val_loss: 1.9834 - val_acc: 0.3047
Epoch 32/50
50000/50000 [==============================] - 6s 115us/step - loss: 1.9816 - acc: 0.3066 - val_loss: 1.9784 - val_acc: 0.3056
Epoch 33/50
50000/50000 [==============================] - 5s 90us/step - loss: 1.9767 - acc: 0.3073 - val_loss: 1.9735 - val_acc: 0.3077
Epoch 34/50
50000/50000 [==============================] - 5s 109us/step - loss: 1.9718 - acc: 0.3104 - val_loss: 1.9687 - val_acc: 0.3088
Epoch 35/50
50000/50000 [==============================] - 6s 124us/step - loss: 1.9671 - acc: 0.3106 - val_loss: 1.9642 - val_acc: 0.3116
Epoch 36/50
50000/50000 [===============

50000/50000 [==============================] - 5s 109us/step - loss: 1.0777 - acc: 0.6151 - val_loss: 1.5404 - val_acc: 0.4891
Epoch 31/50
50000/50000 [==============================] - 5s 109us/step - loss: 1.0610 - acc: 0.6198 - val_loss: 1.6544 - val_acc: 0.4649
Epoch 32/50
50000/50000 [==============================] - 6s 110us/step - loss: 1.0458 - acc: 0.6277 - val_loss: 1.6767 - val_acc: 0.4757
Epoch 33/50
50000/50000 [==============================] - 6s 118us/step - loss: 1.0228 - acc: 0.6324 - val_loss: 1.5797 - val_acc: 0.4894
Epoch 34/50
50000/50000 [==============================] - 6s 111us/step - loss: 1.0154 - acc: 0.6391 - val_loss: 1.7455 - val_acc: 0.4595
Epoch 35/50
50000/50000 [==============================] - 6s 119us/step - loss: 0.9958 - acc: 0.6433 - val_loss: 1.6600 - val_acc: 0.4880
Epoch 36/50
50000/50000 [==============================] - 6s 112us/step - loss: 0.9789 - acc: 0.6490 - val_loss: 1.7419 - val_acc: 0.4700
Epoch 37/50
50000/50000 [==============

...

50000/50000 [==============================] - 5s 108us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 34/50
50000/50000 [==============================] - 6s 113us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 35/50
50000/50000 [==============================] - 5s 105us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 36/50
50000/50000 [==============================] - 5s 105us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 37/50
50000/50000 [==============================] - 5s 98us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 38/50
50000/50000 [==============================] - 6s 123us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 39/50
50000/50000 [==============================] - 6s 115us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 40/50
50000/50000 [=

50000/50000 [==============================] - 5s 106us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 34/50
50000/50000 [==============================] - 5s 110us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 35/50
50000/50000 [==============================] - 5s 101us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 36/50
50000/50000 [==============================] - 6s 124us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 37/50
50000/50000 [==============================] - 6s 110us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 38/50
50000/50000 [==============================] - 6s 120us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 39/50
50000/50000 [==============================] - 6s 120us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 40/50
50000/50000 [

50000/50000 [==============================] - 6s 119us/step - loss: 0.8662 - acc: 0.6915 - val_loss: 1.4857 - val_acc: 0.5222
Epoch 35/50
50000/50000 [==============================] - 6s 118us/step - loss: 0.8418 - acc: 0.7006 - val_loss: 1.5000 - val_acc: 0.5283
Epoch 36/50
50000/50000 [==============================] - 6s 120us/step - loss: 0.8361 - acc: 0.7017 - val_loss: 1.5335 - val_acc: 0.5187
Epoch 37/50
50000/50000 [==============================] - 6s 120us/step - loss: 0.8166 - acc: 0.7088 - val_loss: 1.5706 - val_acc: 0.5184
Epoch 38/50
50000/50000 [==============================] - 6s 121us/step - loss: 0.7939 - acc: 0.7163 - val_loss: 1.5814 - val_acc: 0.5214
Epoch 39/50
50000/50000 [==============================] - 6s 121us/step - loss: 0.7808 - acc: 0.7198 - val_loss: 1.5530 - val_acc: 0.5257
Epoch 40/50
50000/50000 [==============================] - 6s 123us/step - loss: 0.7780 - acc: 0.7233 - val_loss: 1.6159 - val_acc: 0.5196
Epoch 41/50
50000/50000 [==============

50000/50000 [==============================] - 6s 121us/step - loss: 0.8438 - acc: 0.6978 - val_loss: 1.5048 - val_acc: 0.5269
Epoch 36/50
50000/50000 [==============================] - 6s 119us/step - loss: 0.8326 - acc: 0.7019 - val_loss: 1.5491 - val_acc: 0.5219
Epoch 37/50
50000/50000 [==============================] - 6s 120us/step - loss: 0.8016 - acc: 0.7140 - val_loss: 1.5309 - val_acc: 0.5240
Epoch 38/50
50000/50000 [==============================] - 6s 121us/step - loss: 0.7927 - acc: 0.7178 - val_loss: 1.5355 - val_acc: 0.5247
Epoch 39/50
50000/50000 [==============================] - 6s 121us/step - loss: 0.7801 - acc: 0.7214 - val_loss: 1.5832 - val_acc: 0.5272
Epoch 40/50
50000/50000 [==============================] - 6s 122us/step - loss: 0.7591 - acc: 0.7302 - val_loss: 1.5683 - val_acc: 0.5348
Epoch 41/50
50000/50000 [==============================] - 6s 124us/step - loss: 0.7500 - acc: 0.7323 - val_loss: 1.6305 - val_acc: 0.5244
Epoch 42/50
50000/50000 [==============

50000/50000 [==============================] - 8s 154us/step - loss: 0.8514 - acc: 0.6978 - val_loss: 1.5018 - val_acc: 0.5248
Epoch 37/50
50000/50000 [==============================] - 8s 155us/step - loss: 0.8281 - acc: 0.7041 - val_loss: 1.5265 - val_acc: 0.5211
Epoch 38/50
50000/50000 [==============================] - 8s 159us/step - loss: 0.8159 - acc: 0.7113 - val_loss: 1.5315 - val_acc: 0.5310
Epoch 39/50
50000/50000 [==============================] - 8s 162us/step - loss: 0.7941 - acc: 0.7170 - val_loss: 1.5337 - val_acc: 0.5281
Epoch 40/50
50000/50000 [==============================] - 5446s 109ms/step - loss: 0.7728 - acc: 0.7278 - val_loss: 1.6740 - val_acc: 0.4985
Epoch 41/50
50000/50000 [==============================] - 9s 178us/step - loss: 0.7797 - acc: 0.7230 - val_loss: 1.5726 - val_acc: 0.5309
Epoch 42/50
50000/50000 [==============================] - 7s 137us/step - loss: 0.7575 - acc: 0.7306 - val_loss: 1.6175 - val_acc: 0.5280
Epoch 43/50
50000/50000 [===========

In [ ]:
len(results)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

"""
將實驗結果繪出
"""
color_bar = ["r", "g", "b", "y", "m", "k",'black','gray','lime','turquoise','slateblue','mediumslateblue','wheat','darkgoldenrod']
#color_bar = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12']


plt.figure(figsize=(15,15))
for i, cond in enumerate(results.keys()):
    if i<6:
        plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
        plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(15,15))
for i, cond in enumerate(results.keys()):
    if i<6:
        plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
        plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()